In [ ]:
import requests 
import bs4
import zipfile
import re


url = 'https://unsplash.com/'
# url = 'https://collections.louvre.fr/en/'
# url = 'https://scryfall.com/sets/rvr'

# на ресурсах отличается формат хранения, не все содержат домен
match = re.match(r"^https?://[^\/]+", url)
base_url = match.group(0)

response = requests.get(url)
soup = bs4.BeautifulSoup(response.text, 'html.parser')

# собираем все src, до которых можем дотянуться, игнорируя их происхождение.
# от разумного подхода к работе с тегами отказался в пользу универсальности
src_urls = set()
for tag in soup.find_all():
    if tag.has_attr('src'):
        src_urls.add(tag['src'])


def get_jpg(src_url):
    print(src_url)
    response_jpg = requests.get(src_url)    
    if response_jpg.status_code != 200:
        print('Невалидная ссылка, код ответа:', response_jpg.status_code)
    else:
        if not response_jpg.headers['Content-Type'].startswith('image/'):
            print('Не картинка')
        else:
            # Решил отсечь часть изображений по размеру файла, с unsplash тянул аватарки
            size = int(response_jpg.headers.get('Content-Length', 0))
            if size < 5*1024:
                print('Изображение маленькое, игнорируем')
            else: 
                file_name = f'image{i}.jpg'
                zpf.writestr(file_name, response_jpg.content)

with zipfile.ZipFile('images_try2.zip', 'w') as zpf:
    for i, src_url in enumerate(src_urls):
        if '.svg' in src_url:
                print('Игнорируем векторную графику')
        else:
            if 'http' in src_url:
                get_jpg(src_url)
            elif src_url[1] == '/':
                get_jpg(base_url+src_url)
                



        
